In [ ]:
from pdfminer.high_level import extract_text
import cohere
import re
from IPython.display import display, Markdown
import requests
from bs4 import BeautifulSoup

co = cohere.Client("YOUImGXa6Wg0XgfsFdM0c2iIFOX3fM8UDMKzdGYZ")

def main():
    url = "https://nexergroup.teamtailor.com/jobs/5081724-hil-test-engineer"
    filename = "Abhishek-Kolekar-FlowCV-Resume-20241104.pdf"
    textFromResume = extract_text_from_pdf(filename)
    skills_req = extract_skills(url)
    print(skills_req)
    output = skills_match(textFromResume, skills_req)
    display(Markdown(output))

# Extracting text from pdf as is
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def clean_text(text):
    text = re.sub(r'\n+', '\n', text)   # Removes newlines
    text = re.sub(r'\s+', ' ', text)    # Remove multiple spaces
    text = re.sub(r'Page \d+ of \d+', '', text) # Remove page numbers
    text = text.replace('•', ' ')   # Replace bullets with space.
    text = re.sub(r'\+46-\d{9}', lambda x: x.group().replace('-', ''), text) # Remove hyphens or spaces in phone number 
    text = text.strip() # Remove redundant spaces
    return text

def extract_skills(url):
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Failed to retrieve the page.")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')

    job_info = soup.get_text(separator='\n',strip=True)

    if job_info:
        # Remove any extraneous HTML tags and get text content
        response = co.chat(
            message = "Think Carefully. Following is a scrapped webpage of a role I want to apply to:\n" + job_info + "\n<end of webpage>\n\nImagine you are a seasoned hiring manager. List ALL the main keywords you would put in this job description that candidates need to fulfill. Now take out keywords which are synonymous or derivative of each other. Give those keywords a weighing score from 1 to 10 in order of relevance. Feel free to merge similar keywords and aggregate their score. Output the main keywords and their scores as a dictionary, only keep the dictionary, no extra word"
        )
        skills_list = response.text
        return skills_list
    else:
        print("Job description section not found.")
        return None

def skills_match(text, reqs):
    response = co.chat(
        message = "Think Carefully. Following is my resume:\n" + text + "\n<end of resume>\nI am applying for a job which has the keywords and their respective percieved relevance in this dictionary:\n" + str(reqs) + "\n Imagine you're a hiring manager. Please provide a score out of 100 indicating how likely you are to get me to an interview, based on my resume, without any sugarcoating, and elaborate you scoring. Additionally, offer recommendations for changes I can make to my resume to maximize my chances of an interview, excluding any suggestions related to work experience, roles, responsibilities, proofreading, cover letter tailoring, or English proficiency. Use markdown for formatting."
    )
    return response.text

if __name__ == "__main__":
    main()